## integrating llms into my notebook worflows.

im finally giving in and integrating llms into my notebooks. i'll eventually need to experiment with their accessibility. i do believe that llms are assistive technology is a familiar way to interactive computing, one compute rich and the other compute poor. i'll learn to understand this technology as an accessibility tool. it feels like the sharper image selling accessibility to abled people at high markups.

to use on the daily i want this feature to be lazy. 
i'll start using huggingfae and if i dig this life then ill edit this document accordingly.
i want it as a magic and a code fence for `midgy`. i want caching.

In [4]:
    import langchain.cache, langchain.globals, langchain.chains.conversation.base, langchain.chains.conversation.memory, langchain.prompts.prompt
    import langchain_community.chat_models.huggingface

    if "initialize" not in locals():
        @functools.lru_cache
        def initialize():
            langchain.globals.set_llm_cache(langchain.cache.SQLiteCache())
            __import__("dotenv").load_dotenv()
    initialize()

https://python.langchain.com/docs/modules/model_io/chat/chat_model_caching/

In [2]:
%%
create our `llm` using the default `parser` values

    parser = argparse.ArgumentParser(prog="🗩")
    parser.add_argument("repo_id",  type=str, nargs="?", default="mistralai/Mistral-7B-Instruct-v0.2")
    parser.add_argument("-t", "--max_new_tokens",  type=int, default=512)
    parser.add_argument("-k", "--top_k",  type=int, default=30)
    parser.add_argument("-r", "--repetition_penalty", type=float, default=1.03)

ensure that our creditials were accepted.

    >>> registration.stdout
    'Token...Login successful...'

    %%capture registration
    llm = langchain_community.chat_models.huggingface.HuggingFaceEndpoint(**vars(parser.parse_args([])))

i figure it makes sense to curate my own prompt template over time. i'll start with this template and modify the files on disk

https://python.langchain.com/docs/modules/memory/conversational_customization/

```ipython
%%file prompt_template.md
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
The AI always responds in well structured markdown with headings beginning at level 2.

Current conversation:
{history}
🥩: {input}
🤗:
```

establish a conversation chain. we'll want to name these things and curate them over time.

In [5]:
    conversation = langchain.chains.conversation.base.ConversationChain(
        prompt=langchain.prompts.prompt.PromptTemplate(input_variables=["history", "input"], template=Path("prompt_template.md").read_text()),
        llm=llm,
        memory=langchain.chains.conversation.memory.ConversationBufferMemory(ai_prefix="🤗", human_prefix="🥩"),
    )

In [6]:
    async def invoke(input):
        global result
        try:
            id = display(IPython.display.HTML(shell.tangle.parser.parser.render(F"pending\n: > {input}")), display_id=True)
            return id
        finally:
            result = await conversation.ainvoke(input)
            left, sep, right = result["response"].rpartition("🤗:")
            id.update(IPython.display.Markdown(right))

register the magic functions for `chat`

In [7]:
    @functools.partial(shell.register_magic_function, magic_kind="line_cell")
    def chat(line, cell=None):
        if cell:
            asyncio.create_task(invoke(cell))

## conclusion

an overall theme is curating your llm over time. the feeling of control and consent is important in the interactive computing experience.

## some queries i made making this

In [8]:
%%chat
how do i understand css specificity?

## Understanding CSS Specificity

CSS Specificity is a mechanism used by the Cascading Style Sheets (CSS) to determine which style property should be applied to an HTML element when there are multiple conflicting declarations for that property.

Here's a breakdown of how CSS Specificity works:

### Inline Styles

Styles defined directly on the HTML element using the `style` attribute take precedence over all other styles.

```html
<div id="my-div" style="color: red;">This text will be red.</div>

<style>
  #my-div {
    color: blue;
  }
</style>
```
In this example, the inline style on the `div` element with the id `my-div` will override the `color` property in the external stylesheet.

### ID Selector

Selectors that use the `#` symbol to target an element by its id have the highest specificity value, at 1000.

```css
#my-id {
  color: green;
}
```

### Class and Attribute Selectors

Class and attribute selectors have a specificity value of 10.

```css
.my-class {
  color: yellow;
}

[data-custom="value"] {
  color: orange;
}
```

### Type Selectors and Pseudo-Classes

Type selectors (selecting elements based on their tag name) and pseudo-classes have a specificity value of 1.

```css
p {
  color: black;
}

p:hover {
  color: gray;
}
```

### Importance

You can also manually increase the specificity of a rule by adding the `!important` keyword. Be careful with this, as it may make debugging more difficult.

```css
.my-class {
  color: yellow;
}

.my-class {
  color: red !important;
}
```
In this example, the second rule with the `!important` keyword takes precedence over the first rule.

Keep in mind that the order in which the styles are defined also plays a role in determining which one takes effect,

In [9]:
await invoke("what is a tagged pdf and how does it relate to an html document? can tagged pdf be made from well formed html?")

## Tagged PDF and HTML Relationship

A Tagged PDF (PDF/A or Tagged PDF/A) is a type of Portable Document Format (PDF) file that contains structure information (tags) in addition to the visual representation of the document. These tags describe the meaning of the content, allowing assistive technology and search engines to access and understand the information within the PDF.

HTML, on the other hand, is a markup language used for creating the structure, content, and presentation of web pages. HTML files are text-based and consist of a hierarchical tree-like structure with elements and tags.

Regarding your second question, it is technically possible to create a Tagged PDF from well-formed HTML using dedicated tools like Adobe Acrobat or third-party libraries such as iText or Apache PDFBox. However, the process involves converting the HTML to XML, applying XSL-FO stylesheets to transform the XML into Tagged PDF format, and finally exporting the document as a Tagged PDF. This conversion process can be complex, and the resulting Tagged PDF may not perfectly preserve all the formatting and layout of the original HTML document.

<DisplayHandle display_id=1bcf2ddf37a486c47bde440f32c99269>

In [10]:
await invoke("do web content accessibiity guidelines apply to pdf? is there a different standard? links are helpful.")

## Web Content Accessibility Guidelines (WCAG) and PDF

Yes, Web Content Accessibility Guidelines (WCAG) apply to PDF documents to ensure they are accessible to people with disabilities. WCAG provides a set of recommendations for making web content more accessible. Although WCAG is primarily designed for web content, many of its principles can be applied to PDF documents to make them more accessible.

To help you understand how to make PDFs accessible, the World Wide Web Consortium (W3C) has created an additional set of guidelines called the [Accessible PDF Metadata](https://www.w3.org/TR/pdf-accessibility-1/) (APM) specification. APM focuses on providing metadata that describes the structure and accessibility features of a PDF document.

Here are some ways to make PDFs accessible:

1. Use descriptive and meaningful text for all non-text content, such as images, graphs, and charts.
2. Provide alternative text for images using the "Alt Text" tag.
3. Make sure the document is tagged correctly, including headings, lists, tables, and other structural elements.
4. Use proper color contrast for readability.
5. Ensure all form fields are labeled and accessible.
6. Add bookmarks and internal links to help users navigate through the document.
7. Use clear and consistent language and formatting throughout the document.
8. Use a logical reading order.

For more detailed information, you can refer to the [PDF Accessibility Checklist](https://www.adobe.com/content/dam/acom/en/devnet/accessibility/pdfs/pdfs_accessibility_checklist.pdf) provided by Adobe.

By following these guidelines, you can ensure that your PDF documents are accessible to as many users as possible, regardless of their abilities.

<DisplayHandle display_id=3bb2b4603e2d884a4ba367ac1e0b8ebc>

In [ ]:
await invoke("what are the major css apis or concepts like block contexts and 3d rendering?")

In [ ]:
await invoke("could you tell me some more core features of css so that i can better understand ALL of the capabilities")

In [ ]:
await invoke("what are some of the most advanced css concepts a seasoned developer will encounter?")